<a href="https://colab.research.google.com/github/anuj040104/Fake-News-Prediction/blob/main/Fake_News_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing libraries

In [23]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [24]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [25]:
# printing the stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [26]:
!iconv -f ISO-8859-1 -t UTF-8 /content/train.csv > /content/train_utf8.csv

In [27]:
import csv
news_dataset = pd.read_csv('/content/train.csv',quoting=csv.QUOTE_NONNUMERIC)

In [28]:
news_dataset.shape

(20800, 5)

In [29]:
news_dataset.head()

,id,title,author,text,label
0,0.0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1.0
1,1.0,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0.0
2,2.0,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1.0
3,3.0,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1.0
4,4.0,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1.0


In [30]:
news_dataset['label'].value_counts()

1.0    10413
0.0    10387
Name: label, dtype: int64

In [31]:
# 1 -> Fake news
# 0 -> Real news

# counting the number of missing values in the dataset
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [32]:
# replacing the null values with empty string
news_dataset = news_dataset.fillna('')

In [33]:
# merging the author name and news title
news_dataset['content'] = news_dataset['author']+' '+news_dataset['title']
print(news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [34]:
# separating the data & label
X = news_dataset.drop(columns='label', axis=1)
Y = news_dataset['label']

In [35]:
print(X)
print(Y)

            id                                              title  \
0          0.0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1.0  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2.0                  Why the Truth Might Get You Fired   
3          3.0  15 Civilians Killed In Single US Airstrike Hav...   
4          4.0  Iranian woman jailed for fictional unpublished...   
...        ...                                                ...   
20795  20795.0  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796.0  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797.0  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798.0  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799.0                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2              

In [36]:
# Using port stemmer

port_stem = PorterStemmer()

In [37]:
# Defining a function to reduce all words to its root word
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [38]:
# Applying stem function on content column
news_dataset['content'] = news_dataset['content'].apply(stemming)
print(news_dataset['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [39]:
#separating the data and label
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [40]:
print(X)
print(Y)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']
[1. 0. 1. ... 0. 1. 1.]


In [41]:
Y.shape

(20800,)

In [42]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)
print(X)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

Splitting the dataset into training and test data

In [43]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,stratify=Y,test_size=0.2,random_state=2)

Training the model : Logistic Regression

In [44]:
model = LogisticRegression()

In [66]:
model.fit(X_train, Y_train)

LogisticRegression()

Accuracy score of the training dataset

In [46]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.9865985576923076


Accuracy code of the test dataset

In [47]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.9790865384615385


Accuracy score of the test data given in kaggle

In [57]:
news_test = pd.read_csv('/content/test.csv')
news_result = pd.read_csv('/content/submit (1).csv')
print(news_test)
print(news_result['label'])

         id                                              title  \
0     20800  Specter of Trump Loosens Tongues, if Not Purse...   
1     20801  Russian warships ready to strike terrorists ne...   
2     20802  #NoDAPL: Native American Leaders Vow to Stay A...   
3     20803  Tim Tebow Will Attempt Another Comeback, This ...   
4     20804                    Keiser Report: Meme Wars (E995)   
...     ...                                                ...   
5195  25995  The Bangladeshi Traffic Jam That Never Ends - ...   
5196  25996  John Kasich Signs One Abortion Bill in Ohio bu...   
5197  25997  California Today: What, Exactly, Is in Your Su...   
5198  25998  300 US Marines To Be Deployed To Russian Borde...   
5199  25999  Awkward Sex, Onscreen and Off - The New York T...   

                       author  \
0            David Streitfeld   
1                         NaN   
2               Common Dreams   
3               Daniel Victor   
4     Truth Broadcast Network   
...      

In [60]:
# replacing the null values with empty string
news_test = news_test.fillna('')

# merging the author name and news title
news_test['content'] = news_test['author']+' '+news_test['title']
print(news_test['content'])

0       David Streitfeld Specter of Trump Loosens Tong...
1        Russian warships ready to strike terrorists n...
2       Common Dreams #NoDAPL: Native American Leaders...
3       Daniel Victor Tim Tebow Will Attempt Another C...
4       Truth Broadcast Network Keiser Report: Meme Wa...
                              ...                        
5195    Jody Rosen The Bangladeshi Traffic Jam That Ne...
5196    Sheryl Gay Stolberg John Kasich Signs One Abor...
5197    Mike McPhate California Today: What, Exactly, ...
5198     300 US Marines To Be Deployed To Russian Bord...
5199    Teddy Wayne Awkward Sex, Onscreen and Off - Th...
Name: content, Length: 5200, dtype: object


In [61]:
# Applying stem function on content column
news_test['content'] = news_test['content'].apply(stemming)
print(news_test['content'])

0       david streitfeld specter trump loosen tongu pu...
1       russian warship readi strike terrorist near al...
2       common dream nodapl nativ american leader vow ...
3       daniel victor tim tebow attempt anoth comeback...
4        truth broadcast network keiser report meme war e
                              ...                        
5195    jodi rosen bangladeshi traffic jam never end n...
5196    sheryl gay stolberg john kasich sign one abort...
5197    mike mcphate california today exactli sushi ne...
5198                us marin deploy russian border norway
5199        teddi wayn awkward sex onscreen new york time
Name: content, Length: 5200, dtype: object


In [62]:
print(news_test)

         id                                              title  \
0     20800  Specter of Trump Loosens Tongues, if Not Purse...   
1     20801  Russian warships ready to strike terrorists ne...   
2     20802  #NoDAPL: Native American Leaders Vow to Stay A...   
3     20803  Tim Tebow Will Attempt Another Comeback, This ...   
4     20804                    Keiser Report: Meme Wars (E995)   
...     ...                                                ...   
5195  25995  The Bangladeshi Traffic Jam That Never Ends - ...   
5196  25996  John Kasich Signs One Abortion Bill in Ohio bu...   
5197  25997  California Today: What, Exactly, Is in Your Su...   
5198  25998  300 US Marines To Be Deployed To Russian Borde...   
5199  25999  Awkward Sex, Onscreen and Off - The New York T...   

                       author  \
0            David Streitfeld   
1                               
2               Common Dreams   
3               Daniel Victor   
4     Truth Broadcast Network   
...      

In [63]:
# converting the textual data to numerical data
x = news_test['content'].values
vectorizer.fit(x)

x = vectorizer.transform(x)
print(x)

  (0, 9086)	0.08756899768504829
  (0, 8618)	0.2951833364547817
  (0, 8391)	0.11042633230471807
  (0, 8258)	0.35551158241433845
  (0, 8217)	0.08565162006825207
  (0, 7825)	0.35551158241433845
  (0, 7815)	0.3392484573767603
  (0, 7632)	0.35551158241433845
  (0, 7416)	0.29990759064553757
  (0, 6435)	0.35551158241433845
  (0, 5528)	0.0847731814904578
  (0, 4746)	0.35551158241433845
  (0, 1960)	0.20372848702181867
  (1, 8826)	0.4600630644689105
  (1, 8135)	0.35260162515090443
  (1, 7823)	0.36689240142839846
  (1, 6999)	0.30028557064663775
  (1, 6583)	0.40045820302192875
  (1, 5485)	0.37981022352550325
  (1, 186)	0.36689240142839846
  (2, 8964)	0.295064720779198
  (2, 8765)	0.28434502467517675
  (2, 7741)	0.2982257606996168
  (2, 6196)	0.2207192031076798
  (2, 5596)	0.3312058036253688
  :	:
  (5196, 795)	0.21762533251043967
  (5196, 24)	0.2798172726761391
  (5197, 9086)	0.12439214769277508
  (5197, 8237)	0.3546424451054057
  (5197, 8217)	0.1216685043258708
  (5197, 7942)	0.46551291399479516


In [68]:
# accuracy score on the test data taken from kaggle (Fake News)

news_test_prediction = model.predict(x)
news_data_accuracy = accuracy_score(news_test_prediction, news_result['label'].values)
print('Accuracy score of the test data : ', news_data_accuracy)

Accuracy score of the test data :  0.8371153846153846
